In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd  # This is a duplicate, but included for completeness
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/shazizisazi/Datasets/refs/heads/main/tweet_emotions.csv").drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# Define text preprocessing functions

def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    
    # Split the input text string into a list of words
    text = text.split()
    
    # Apply lemmatization to each word
    text = [lemmatizer.lemmatize(word) for word in text]
    
    # Join the words back into a single string
    return " ".join(text)

# ---

def remove_stop_words(text):
    """Remove stop words from the text."""
    # Load the set of English stop words
    stop_words = set(stopwords.words("english"))
    
    # Split text into words and filter out stop words
    text = [word for word in str(text).split() if word not in stop_words]
    
    # Join the remaining words
    return " ".join(text)

# ---

def removing_numbers(text):
    """Remove numbers from the text."""
    # Join characters that are NOT digits
    text = "".join([char for char in text if not char.isdigit()])
    return text

# ---

def lower_case(text):
    """Convert text to lower case."""
    # Split text into words
    text = text.split()
    
    # Convert each word to lower case
    text = [word.lower() for word in text]
    
    # Join the lowercased words
    return " ".join(text)

# --- Example Usage ---
# sample_text = "ThE quick brown Foxes ran 10 miles. IS this a good test?"

# text_1 = lower_case(sample_text)
# print(f"1. Lower Case: {text_1}") 

# text_2 = removing_numbers(text_1)
# print(f"2. Removed Numbers: {text_2}") 

# text_3 = remove_stop_words(text_2)
# print(f"3. Removed Stop Words: {text_3}") 

# text_4 = lemmatization(text_3)
# print(f"4. Lemmatized: {text_4}")

def removing_punctuations(text):
    """Remove punctuations from the text."""
    # This line seems to have a typo or custom formatting (%s, % re.escape...) 
    # based on the image, which makes it hard to reproduce perfectly.
    # The standard way to remove punctuations using regex is simpler:
    
    # 1. Remove all standard punctuation using re.sub
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    
    # 2. Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text)
    
    # 3. Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# ---

def removing_urls(text):
    """Remove URLs from the text."""
    # Define a regular expression pattern for matching URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    
    # Substitute the matched URLs with an empty string ('r' is likely a placeholder for '')
    return url_pattern.sub('', text)

# ---

def normalize_text(df):
    """Normalize the text data in the 'content' column of a DataFrame."""
    try:
        # NOTE: Assumes functions like lower_case, remove_stop_words, 
        # removing_numbers, removing_punctuations, removing_urls, and lemmatization are defined.
        
        # 1. Convert to lower case
        df['content'] = df['content'].apply(lower_case)
        
        # 2. Remove stop words
        df['content'] = df['content'].apply(remove_stop_words)
        
        # 3. Remove numbers
        df['content'] = df['content'].apply(removing_numbers)
        
        # 4. Remove punctuations
        df['content'] = df['content'].apply(removing_punctuations)
        
        # 5. Remove URLs
        df['content'] = df['content'].apply(removing_urls)
        
        # 6. Lemmatize
        df['content'] = df['content'].apply(lemmatization)
        
        return df
        
    except Exception as e:
        print(f"Error during text normalization: {e}")
        # The original code has 'raise', meaning it stops execution and re-raises the exception
        raise

# --- Example of creating a placeholder function for demonstration ---
def lower_case(text): return text.lower()
def remove_stop_words(text): return text # Placeholder
def removing_numbers(text): return "".join([char for char in text if not char.isdigit()])
def lemmatization(text): return text # Placeholder

# --- Example Usage (requires pandas) ---
# data = {'content': ["THIS is a test string! Check out this link: https://example.com/page-1", "Another string with NUMBERS 123"]}
# df = pd.DataFrame(data)
# 
# df_normalized = normalize_text(df.copy())
# print(df_normalized)

In [5]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue i know i was listenin to bad habit ...
1,sadness,layin n bed with a headache ughhhhwaitin on yo...
2,sadness,funeral ceremonygloomy friday
3,enthusiasm,wants to hang out with friends soon
4,neutral,dannycastillo we want to trade with someone wh...


In [6]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_17716\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed with a headache ughhhhwaitin on yo...
2,0,funeral ceremonygloomy friday
6,0,i should be sleep but im not thinking about an...
8,0,charviray charlene my love i miss you
9,0,kelcouch im sorry at least its friday


In [9]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/himanshu7-t/mlops-mini-project.mlflow')
dagshub.init(repo_owner='himanshu7-t', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_experiment("Logistic Regression Baseline")

Accessing as himanshu7-t

Initialized MLflow to track repo "himanshu7-t/mlops-mini-project"

Repository himanshu7-t/mlops-mini-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/b37d89bf265d4e95a6a6587ebc60b338', creation_time=1759470377014, experiment_id='0', last_update_time=1759470377014, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
with mlflow.start_run():
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model (workaround for DagsHub)
    import joblib
    joblib.dump(model, "log_reg_model.pkl")
    mlflow.log_artifact("log_reg_model.pkl")

    # Log notebook (just upload, don’t re-execute)
    notebook_path = "exp1_baseline_model.ipynb"
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


Accuracy: 0.7985542168674699
Precision: 0.7792329279700655
Recall: 0.8206896551724138
F1 Score: 0.7994241842610365
🏃 View run suave-wasp-950 at: https://dagshub.com/himanshu7-t/mlops-mini-project.mlflow/#/experiments/0/runs/610c0800c40f45868466b1398b7244cc
🧪 View experiment at: https://dagshub.com/himanshu7-t/mlops-mini-project.mlflow/#/experiments/0
